## Configuration interaction
### General theory

One of the conceptually (but not computationally) simplest way to solve the Schrödinger equation is to simply expand our many-body wavefunction on a n-electron basis set (not to be confused with the 1-electron basis functions used to expand orbitals):

$$| \Psi_{CI} \rangle = \sum_k c_k | \mathbf{k} \rangle $$ 

with energy:

$$ E = \frac{\langle \Psi_{CI} | H | \Psi_{CI} \rangle}{\langle \Psi_{CI} | \Psi_{CI} \rangle} $$

We will discuss in the next section the choice of the basis, but in all cases, as the size of this basis set increases, the solution converges to the exact solution of the Schrödinger equation.

There is a redundancy in the parameters $c_k$ as multiplying them all by a constant does not change the energy. We thus add the constraint that the wavefunction should be normalised $\langle \Psi_{CI} | \Psi_{CI} \rangle = \sum_k c_k^2 = 1$. To minimize the wavefunction subject to this constraint, we write a Lagrangian:

$$ L = E - \epsilon(\sum_k c_k^2 -1) $$

To define our minimisation we set the gradient of our Lagrangian to 0 and obtain:

$$ \frac{dL}{d c_k} =  \sum_i \langle \mathbf{i} | H | \mathbf{k} \rangle c_i - \epsilon c_k + cc.= 0 $$

with $cc.$ the complex conjugate of the expression. This can finally be recast in the familiar form:

$$ \mathbf{H} \mathbf{c} = \epsilon \mathbf{c} $$

showing that minimising the wavefunction is equivalent to diagonalising the Hamiltonian matrix in our chosen basis, with the eigenvalues being the energies of all possible states in the system. The ground state simply corresponds to the lowest eigenvalue.

### The n-electron basis

There are in principle an infinite number of possible n-electron basis to choose from, but we ideally want the expansion to converge quickly (compact expansion) and the functions to be easily manipulatable to quickly compute the Hamiltonian elements. Naturally, these two criteria often go opposite.

As electron correlation depends on the distance between electrons, it would be natural to use functions that depend explicitely on this distance, the same way as our gaussian or Slater 1-electron basis depends explicitely on the distance between electron and nuclei. Wavefunction having this properties are typically called explicitly correlated wavefunctions, but this interelectronic distance makes the calculation of integrals very difficult as it couples multiple electrons together.

We often drop this explicit dependence and instead use a wavefunction that is simply formed from products of one-electron functions, i.e. product of orbitals. The simplest of these basis functions if the Hartree product, but since we know the wavefunction should be antisymmetric with respect to the exchange of two electrons, we instead use the antisymmetrised version of the Hartree product: Slater determinants (SD). As each SD represents a different electronic configuration, the resulting method is called configuration interaction (CI). As discussed in the previous sections, this basis of SD constitutes a complete basis within a specified one-electron basis set, and thus the solution obtained by minimizing the energy of this wavefunction is the exact ground state within that one-electron basis.

In some codes, configuration state functions (CSF) are used instead of SDs. CSF are fixed linear combinations of SD but with the attractive properties that they are also eigenfunctions of the $S^2$ spin operator. This spin constraints means that the resulting wavefunction will always be spin-adapted and also that CSF are less numerous than Slater Determinants which has advantages on a computational standpoint. However, SD can be expressed as ordered "strings" of $\alpha$ and $\beta$ creating operators, and this separation of $\alpha$ and $\beta$ electrons allows to write efficient computer implementation, outweighing the benefits of the slightly smaller expansion of CSF.

Having chosen SD as our basis functions, we now need an efficient way to create our expansion, i.e. a direct link between a given index in our expansion and the corresponding occupation of the SD. This link should work both ways, matching a given occupation to the corresponding index in the expansion. A common choice is the so-called lexical ordering or the reverse lexical ordering. The details are beyond the scope of this page, but fortunately, the MultiPsi package provide the tools to do this.

Let's start by initialising a calculation for O$_2$. We will first run a Hartree-Fock calculation to generate starting orbitals. The ground state of this molecule is a triplet, but for the Hartree-Fock step we will simply run a singlet closed-shell. This is not an issue as full CI is exact and does not depend on the actual shape of the orbitals.

In [1]:
import veloxchem as vlx
import multipsi as mtp
import numpy as np
O2_xyz=h2o_xyz = """2
water                                                                                                                          
O    0.000000000000        0.000000000000       -0.600000000000 
O    0.000000000000        0.000000000000        0.600000000000 
"""

molecule = vlx.Molecule.from_xyz_string(O2_xyz)
print(molecule.get_string())

basis = vlx.MolecularBasis.read(molecule,"STO-3G")
print(basis.get_string(molecule))

scfdrv = vlx.ScfRestrictedDriver()
scfdrv.compute(molecule, basis)

* Warning * Environment variable OMP_NUM_THREADS not set.
* Warning * Setting OMP_NUM_THREADS to 4.
Molecular Geometry (Angstroms)

  Atom         Coordinate X          Coordinate Y          Coordinate Z  

  O           0.000000000000        0.000000000000       -0.600000000000
  O           0.000000000000        0.000000000000        0.600000000000


Molecular Basis (Atomic Basis)

Basis: STO-3G                                         

  Atom Contracted GTOs          Primitive GTOs           

  O   (2S,1P)                  (6S,3P)                  

Contracted Basis Functions : 10                       
Primitive Basis Functions  : 30                       


                                                                                                                          
                                            Self Consistent Field Driver Setup                                            
                                                                                  

               (   2 O   1p0 :    -0.61)                                                                                  
                                                                                                                          
               Molecular Orbital No.   7:                                                                                 
               --------------------------                                                                                 
               Occupation: 2.0 Energy:   -0.52415 au                                                                      
               (   1 O   1p-1:    -0.66) (   2 O   1p-1:    -0.66)                                                        
                                                                                                                          
               Molecular Orbital No.   8:                                                                                 
               -

Now we can switch the multiplicity back to a triplet.

In [2]:
molecule.set_multiplicity(3)

The next step is to define our CI. The meaning of this step will become more clear when we discuss restricted CI and active spaces. Now we simply define a full CI by using the FCI keyword. Then we use the CIExpansion class to create the tools necessary to handle the CI expansion.

In [7]:
space=mtp.MolSpace(molecule,scfdrv.mol_orbs)
space.FCI()
expansion=mtp.CIExpansion(space)
print(expansion)


          CI expansion:
          -------------
Number of determinants:      1200




Printing the expansion shows the number of Slater determinants. As you can see, the number is already fairly large. This is not an issue in a real calculation, but for testing purposes, we will restrict this number a bit. For this, we will simply excluse the 1s and 2s of the oxygen from the calculation by defining an active space. There are 8 electrons and 6 orbitals remaining, so we will define it as a CAS(8,6): 

In [9]:
space.CAS(8,6)
expansion=mtp.CIExpansion(space)
print(expansion)


          CI expansion:
          -------------
Number of determinants:      120




This number is much more manageable.

Now we can look at the determinants themselves. The easiest way is to use the generator detlist() of the expansion class in a python for loop, which as the name implies will provide the determinants in order, here in reverse lexical ordering. By printing the determinant directly, we get the string representation, with a character for each orbital in the system. '2' indicates that the orbital is doubly occupied, 'a' and 'b' indicates an $\alpha$ or $\beta$ electron, and '0' an empty orbital. The determinant also provides a python list of the orbitals containing a $\alpha$ or $\beta$ electron.

In [10]:
for det in expansion.detlist():
    print(det,det.occ_alpha(),det.occ_beta())

222aa0 [0, 1, 2, 3, 4] [0, 1, 2]
22a2a0 [0, 1, 2, 3, 4] [0, 1, 3]
2a22a0 [0, 1, 2, 3, 4] [0, 2, 3]
a222a0 [0, 1, 2, 3, 4] [1, 2, 3]
22aa20 [0, 1, 2, 3, 4] [0, 1, 4]
2a2a20 [0, 1, 2, 3, 4] [0, 2, 4]
a22a20 [0, 1, 2, 3, 4] [1, 2, 4]
2aa220 [0, 1, 2, 3, 4] [0, 3, 4]
a2a220 [0, 1, 2, 3, 4] [1, 3, 4]
aa2220 [0, 1, 2, 3, 4] [2, 3, 4]
22aaab [0, 1, 2, 3, 4] [0, 1, 5]
2a2aab [0, 1, 2, 3, 4] [0, 2, 5]
a22aab [0, 1, 2, 3, 4] [1, 2, 5]
2aa2ab [0, 1, 2, 3, 4] [0, 3, 5]
a2a2ab [0, 1, 2, 3, 4] [1, 3, 5]
aa22ab [0, 1, 2, 3, 4] [2, 3, 5]
2aaa2b [0, 1, 2, 3, 4] [0, 4, 5]
a2aa2b [0, 1, 2, 3, 4] [1, 4, 5]
aa2a2b [0, 1, 2, 3, 4] [2, 4, 5]
aaa22b [0, 1, 2, 3, 4] [3, 4, 5]
222a0a [0, 1, 2, 3, 5] [0, 1, 2]
22a20a [0, 1, 2, 3, 5] [0, 1, 3]
2a220a [0, 1, 2, 3, 5] [0, 2, 3]
a2220a [0, 1, 2, 3, 5] [1, 2, 3]
22aaba [0, 1, 2, 3, 5] [0, 1, 4]
2a2aba [0, 1, 2, 3, 5] [0, 2, 4]
a22aba [0, 1, 2, 3, 5] [1, 2, 4]
2aa2ba [0, 1, 2, 3, 5] [0, 3, 4]
a2a2ba [0, 1, 2, 3, 5] [1, 3, 4]
aa22ba [0, 1, 2, 3, 5] [2, 3, 4]
22aa02 [0,

### The CI Hamiltonian

Now that we have the list of determinants, we only need to form the CI Hamiltonian. For this, we need to be able to calculate the matrix elements.

MultiPsi also provides a function to do that. For this we need to initialise the CIHam class, which computes the  integrals in the molecular basis and provides several functions to use them in a CI calculation.

In [11]:
CIham=mtp.CIHam(molecule,basis,expansion) #Contains integrals and functions using them
CIham.compute_ints()

We can now compute the Hamiltonian. The easiest is to simply do a double loop over SD and then computing the matrix element using the Hij function of CIHam:

In [12]:
Ham=np.zeros((expansion.nDet,expansion.nDet))
for i,idet in enumerate(expansion.detlist()):
    for j,jdet in enumerate(expansion.detlist()):
        Ham[i,j]=CIham.Hij(idet,jdet)

If we diagonalise this hamiltonian, we now obtain all the energies of the system, both ground and excited states:

In [13]:
w,v=np.linalg.eigh(Ham)
print(w)

[-147.72142568 -147.49304166 -147.49304166 -147.48807549 -147.38735867
 -147.38735867 -147.30200521 -147.27544172 -147.27544172 -147.14731012
 -147.14731012 -147.08159062 -147.08159062 -147.07878345 -147.07001284
 -147.0603461  -147.0603461  -147.00846106 -147.00846106 -146.99566437
 -146.99566437 -146.97297096 -146.97297096 -146.94721035 -146.93555352
 -146.93555352 -146.92638429 -146.88519558 -146.88519558 -146.88031316
 -146.83656166 -146.83656166 -146.81941808 -146.74951722 -146.74951722
 -146.74448247 -146.74448247 -146.73931585 -146.73807678 -146.73807678
 -146.71308893 -146.71308893 -146.69767332 -146.69767332 -146.65702459
 -146.63644112 -146.63644112 -146.56808065 -146.5347469  -146.49721329
 -146.49721329 -146.49222217 -146.48988296 -146.48988296 -146.43557546
 -146.43557546 -146.43157041 -146.41342431 -146.3942147  -146.36844191
 -146.36844191 -146.3414047  -146.33447136 -146.33447136 -146.27735345
 -146.27735345 -146.27295046 -146.2602203  -146.22089315 -146.21143534
 -146.

Note how the first (and lowest) energy is indeed lower than the one found at the Hartree-Fock level. This comes both from electron correlation and from using the correct triplet multiplicity.

It is important to note at that point that some of the states obtained here are not actually triplet but have a higher multiplicity. As we discussed in the previous section, the basis of Slater Determinant is not spin-adapted.  The only constraint in our expansion is the value of $m_s$, here $m_s = 1$, meaning we have two more $\alpha$ electron than $\beta$. This excluses singlet (which can only have $m_s=0$) but does not exclude some quintets or higher.

You may be curious as to what happens in the Hij function, that is, how we compute the $\langle i | H | j \rangle $ matrix elements. The answer to this is the Slater-Condon rules. This rules define the matrix elements in terms of molecular integrals, depending on the difference in occupation between the 2 determinants. The difference in occupation is easily formulated in terms of excitations, with different rules depending on if $|j \rangle$ is singly or doubly excited compared to $|i \rangle$. Any higher excitation will simply lead to 0.

These rules are relatively easy to derive, both in standard and second quantization formulation. The idea is that the Hamiltonian has to "reconcile" the excitations. Being a two-electron operator, the Hamiltonian can reconcile at most two excitations.

In [14]:
def SC_diag(occa, occb):
    Hij=0
    Hij+=CIham.inEne #Inactive energy (inc. nuclear repulsion)
    for i in occa:
        Hij+=CIham.Ftu[i,i] #1-e term = inactive Fock matrix
        for j in occa:
            if i<j:
                Hij+=CIham.tuvw[i,i,j,j]-CIham.tuvw[i,j,j,i] #Coulomb-Exchange
        for j in occb:
            Hij+=CIham.tuvw[i,i,j,j]
    for i in occb:
        Hij+=CIham.Ftu[i,i]
        for j in occb:
            if i<j:
                Hij+=CIham.tuvw[i,i,j,j]-CIham.tuvw[i,j,j,i]
    return Hij
def SC_1exc(i,a,ss_occ, os_occ):
    Hij=0
    Hij+=CIham.Ftu[i,a]
    for k in ss_occ:
        Hij+=CIham.tuvw[i,a,k,k]-CIham.tuvw[i,k,k,a]
    for k in os_occ:
        Hij+=CIham.tuvw[i,a,k,k]
    return Hij
def SC_ss1exc(i,a,j,b):
    return CIham.tuvw[i,a,j,b]-CIham.tuvw[i,b,j,a]
def SC_os1exc(i,a,j,b):
    return CIham.tuvw[i,a,j,b]

The difficult part is to make sure that the occupations of the Slater determinants are brought to maximum concordance. The expansion produced by lexical ordering may not have this property, but one can switch electrons until we reach this maximum concordance, introducing a negative phase factor at each electron exchange. Again, MultiPsi provides a convenient way to achieve this. The idea is to explicitely created the single excitation (or double by applying it twice) and then keeps track of the number of exchanges to bring it to natural ordering. The excite_alpha and excite_beta functions do this, providing both the excited determinant and the phase factor.

One can now form the Hamiltonian as described below:

In [15]:
Ham=np.zeros((expansion.nDet,expansion.nDet))
for idet,det in enumerate(expansion.detlist()):
    #Diagonal term
    Ham[idet,idet]=SC_diag(det.occ_alpha(),det.occ_beta())
    #Single excitations alpha
    for i in det.occ_alpha():
        for a in det.unocc_alpha():
            phase,excdet=det.excite_alpha(i,a)
            Ham[idet,excdet.index()]=phase*SC_1exc(i,a,det.occ_alpha(),det.occ_beta())
            #alpha-alpha excitation
            for j in det.occ_alpha():
                if i==j:
                    continue
                for b in det.unocc_alpha():
                    if a==b:
                        continue
                    phase2,exc2det=excdet.excite_alpha(j,b)
                    Ham[idet,exc2det.index()]=phase*phase2*SC_ss1exc(i,a,j,b)
            #alpha-beta excitation
            for j in det.occ_beta():
                for b in det.unocc_beta():
                    phase2,exc2det=excdet.excite_beta(j,b)
                    Ham[idet,exc2det.index()]=phase*phase2*SC_os1exc(i,a,j,b)
    #Single excitations beta
    for i in det.occ_beta():
        for a in det.unocc_beta():
            phase,excdet=det.excite_beta(i,a)
            Ham[idet,excdet.index()]=phase*SC_1exc(i,a,det.occ_beta(),det.occ_alpha())
            #beta-beta excitation
            for j in det.occ_beta():
                if i==j:
                    continue
                for b in det.unocc_beta():
                    if a==b:
                        continue
                    phase2,exc2det=excdet.excite_beta(j,b)
                    Ham[idet,exc2det.index()]=phase*phase2*SC_ss1exc(i,a,j,b) 

The advantage of this loop structure is that we only compute the non-zero elements. It can be easily verified that we obtain the same result:

In [16]:
w,v=np.linalg.eigh(Ham)
print(w[0:3])

[-147.72142568 -147.49304166 -147.49304166]


### Direct CI